# Ejercicios UD03_02

## Clasificar preguntas

En la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) hemos visto el proceso para convertir un texto en una representación numérica que pueda ser utilizada por un algoritmo de aprendizaje automático. Hemos visto diferentes representaciones como *Bolsa de palabras* (BoW) y *incrustaciones de palabras* (word embeddings) y cómo entrenar una red neuronal para clasificar texto.

En esta práctica, deberá repetir el proceso para clasificar las preguntas en temas. Usaremos el conjunto de datos `Trec` que contiene preguntas en inglés y su tema. El conjunto de datos está disponible en [trec](https://huggingface.co/datasets/CogComp/trec).

### Objetivos de la práctica
* Reproducir el proceso visto en la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) para clasificar preguntas en temáticas.
* Deberá preparar una red neuronal con PyTorch para clasificar las preguntas.
* Pruebe las diferentes representaciones vistas para convertir el texto en una representación numérica.
* Tendrá que comparar los resultados obtenidos con las diferentes representaciones.

In [ ]:
# Instalamos las librerías necesarias en las versiones correctas

# %pip install --upgrade torch datasets scikit-learn transformers

In [ ]:
from datasets import load_dataset

# Cargamos el conjunto de datos. Se descargará y almacenará automáticamente en local.
# Este conjunto de datos contiene noticias de diferentes categorías. En este caso
# usaremos las categorías de mundo, deportes, negocios y ciencia ficción/tecnología.

dataset = load_dataset('trec')

dataset

In [ ]:
print(dataset['train'][0])

print(dataset['train'].features)

classes = dataset['train'].features["coarse_label"].names
classes

In [ ]:
# Separar el conjunto de datos en entrenamiento y test
ds_train = dataset['train']
ds_test = dataset['test']
# Veamos cuántos ejemplos hay en cada set
print('Número de ejemplos de train:', len(ds_train))
print('Número de ejemplos de test:', len(ds_test))

In [ ]:
# Imprimimos los primeros 5 ejemplos del conjunto de entrenamiento
for w in ds_train.take(5):
    print(f"{w['coarse_label']} ({classes[w['coarse_label']]}) -> {w['text']}")

In [ ]:
# Utilizamos el tokenizador de Bert (uno de los primeros modelos de lenguaje basados ​​en transformación) para tokenizar las oraciones
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Podiamos ver el vocabulario de tokenización
vocab = tokenizer.get_vocab()
print(len(vocab))

Funcion para convertir nuestra cadena tokenizada a números

In [ ]:
def encode(text):
    tk = tokenizer.tokenize(text)
    return tokenizer.convert_tokens_to_ids(tk)

## BoW
Funcion para calcular el vector BoW de un comentario del dataset

In [ ]:
import torch

len_vocab = len(vocab)

def to_bow(text, tamany_vocabulari=len_vocab):
    res = torch.zeros(tamany_vocabulari, dtype=torch.float32)

    for i in encode(text):
        if i<tamany_vocabulari:
            res[i] += 1
    return res

print(ds_train[0])
print(to_bow(ds_train[0]["text"]))

Función que convierte las palabras textuales en tensores BoW

In [ ]:
from torch.utils.data import DataLoader

def bowify(batch):
    etiquetas = torch.LongTensor([comentario["coarse_label"] for comentario in batch])
    comentarios = torch.stack([to_bow(comentario["text"]) for comentario in batch])

    return (
            etiquetas,
            comentarios
    )

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=bowify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=bowify)

## Red neuronal
El tamaño del vector de entrada es el tamaño del vocabulario, el tamaño de salida corresponde con el número de clases, en este caso 6

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Linear(len(vocab), 6),
    torch.nn.LogSoftmax(dim=1)
)

## Entrenamiento del modelo

In [ ]:
def train_epoch(
    net,
    dataloader,
    lr=0.01,
    optimizer=None,
    loss_fn=torch.nn.NLLLoss(),
    epoch_size=None,
    report_freq=50,
):

    # Si no se especifica un optimizador, usamos Adam
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)

    # Ponemos la red en modo de entrenamiento.Esto activa el comportamiento de las capas de DropOut, por ejemplo.
    net.train()

    # Inicializar las variables que nos servirán para calcular la precisión
    total_loss, acc, count, i = 0, 0, 0, 0

    # Iteremamos sobre el dataloader
    for labels, features in dataloader:

        # Ponemos los gradientes a cero
        optimizer.zero_grad()

        # calculamos la salida de la red
        out = net(features)

        # Calculamos la pérdida. Esta función ya se aplica a Softmax a la salida.
        loss = loss_fn(out, labels)  # cross_entropy(out,labels)

        # Propagamos la pérdida de regreso. Esto hará que se calculen los gradientes .
        loss.backward()

        # Actualizamos los pesos de la red. Esto toma un paso de optimización.
        optimizer.step()

        # Actualizamos variables para calcular la precisión.
        total_loss += loss

        # Calculamos la precisión. Para hacer esto, debemos convertir la salida de red en etiquetas.
        # La clase con la mayor probabilidad es la que predecimos como etiqueta.
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()

        # Actualizamos el contador de muestras
        count += len(labels)

        # Mostramos la precisión cada report_freq muestras
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item()/count}")

        # Si se especifica epoch_size y ya hemos procesado este número de muestras, dejamos el bucle.
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count

num_epochs = 5

for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}")
    train_epoch(net, train_loader, epoch_size=5452)
# train_epoch(net, train_loader, epoch_size=5452)

## Words2Vec

In [ ]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

## Clasificación Word2Vec
Función que recibe un texto y devuelve un vector con la representación w2v del texto

In [ ]:
def to_w2v(text):
    res = torch.zeros(300, dtype=torch.float32)
    for word in text:
        if word in w2v:
            res += torch.tensor(w2v.get_vector(word))
    return res

print(to_w2v(ds_train[0]["text"]))

Función que convierte los datos textales en tensores w2v

In [ ]:
def w2vify(batch):
    etiquetes = torch.LongTensor([comentario["coarse_label"] for comentario in batch])
    comentarios = torch.stack([to_w2v(tokenizer.tokenize(comentario["text"])) for comentario in batch])
    return etiquetes, comentarios

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=w2vify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=w2vify)

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Linear(300, 6),
    torch.nn.LogSoftmax(dim=1)
)

In [ ]:
train_epoch(net, train_loader, epoch_size=5452)

Como podemos observar los resultados son ligeramente mejores con BoW que con Words2Vec